In [ ]:
# Goal: Analyze data on patients with diabetes to see how successful models are at predicting
# hospital readmission within 30 days
#
# Predictors include:
#   time in hospital
#   number of lab procedures, other procedures, and medications given
#   number of diagnoses
#   number of outpatient, emergency, and inpatient visits in the past year
#   demographics: age, race, sex
#   status at discharge (e.g., discharged to same/higher level of care, or discharged home/lower level of care, or left against medical advice)
#   admission type (emergency, urgent, elective, etc)
#   if there was a change in diabetic medications (change, no change)
#   if diabetic medication was prescribed (Y/N)
#   labs
#
# Data Source: https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008
#
# Part 1: Data Cleaning, Preparation, Exploration
# Date last updated: 1/19/2024
# Author: Kay Rubio

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate

In [4]:
# Import data
df = pd.read_csv("./diabetes+130-us+hospitals+for+years+1999-2008/diabetic_data.csv")

In [5]:
# Check data
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [6]:
# Check all column names
column_names = df.columns.tolist()
print("Column Names:", column_names)

Column Names: ['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted']


In [ ]:
# Examination of the potential usefulness of predictors based on theoretical relatedness to hospital
# readmission, and looking at the way they are categorized in the data
"""
race - likely related, will examine further
gender - likely related, will examine further
age - categories for each decade of life.  Concerning that age 0-10 are grouped together when babies medical
        needs are probably quite different from other children. Also, if finding non-linear relationships (e.g., 
        youngest and oldest are highest risks of readmission) may need to trim dataset to only adults for the
        logistic regression
weight - mixed feelings about the usefulness of this since there isn’t enough info to calculate BMI, and some 
        participants are babies and children.  Will leave in for now but consider removing if no significant 
        relationhip to outcome
admission_type_id - will collapse 5, 6, and 8 as missing, but otherwise these categories seem OK
discharge_disposition_id
  - will check for 11, 19, 20, 21 no one should be expired
  - might want to collapse remaining categories: 
    - discharged to lower level of care: 1, 6, 8, 12, 13, 14, 16, 17, 
    - discharged to another inpatient facility: 2, 3, 4, 5, 9, 10, 15, 22, 23, 24, 28, 29
    - left against medical advice: 7 - see if there's a lot of these?
    - not enough information to determine if lower or higher level of care: 18, 25, 26, 30, 27
admission_source_id - There’s a lot of categories, but hard to collapse into things that would be relevant. 
        Some are related to babies being born sick, premie or normal, which is relevant only to babies.  
        Some referrals are clearly from another inpatient facility which could be relevant, but from other 
        categories it’s hard to tell if pt came from another inpatient or outpatient level of care? Plus don’t 
        have data on how long pt was in inpatient care before arrived at current care center.  Will probably remove.
        Or if split for 3 separate analyses on adults vs. children vs. babies dataset, could use for just the babies.
time_in_hospital - likely relevant
payer_code - will remove as unlikely to be related to outcome
medical_specialty - will remove, a lot of missingness and unlikely to be related to outcome
num_lab_procedures - likely relevant
num_procedures - likely relevant
num_medications - likely relevant
number_outpatient, number_emergency, number_inpatient = num of visits of various types in the past year. 
        Wish I had info on insurance status and social class here, which would affect these in addition to pt’s 
        medical condition, but will leave in model as likely relevant.
diag_1, diag_2, diag_3 - Ideally, I’d ask a medical professional to give me some useful categories here to 
        collapse into relevant categories, such as flagging relevant additional diagnoses or severity, but 
        since I don’t have medical advice on hand, going to remove these from this analysis. 
number_diagnoses - likely relevant
various labs - likely relevant (max_glu_serum, A1Cresult, metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide, glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone, tolazamide, examide, citoglipton, insulin, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, metformin-pioglitazone)
change = if meds were changed during hosp, could be relevant?  See if any correlation with outcome
diabetesMed = if diabetic meds were prescribed - likely relevant
"""

In [7]:
# Examine categories in outcome and make it binary. Currently coded as 3 categories:
# No hospital readmission, hospital readmission in >30 days, and hospital readmission in <30 days

# Create a frequency table using value_counts()
ft_readmitted = df['readmitted'].value_counts()

# Calculate the percentage and add it as a new column
ft_readmitted_percent = ft_readmitted.to_frame(name='Frequency')
ft_readmitted_percent['Percentage'] = (ft_readmitted_percent['Frequency'] / len(df)) * 100
print(ft_readmitted_percent)

            Frequency  Percentage
readmitted                       
NO              54864   53.911916
>30             35545   34.928169
<30             11357   11.159916


In [8]:
# Only 11% of patients in dataset were readmitted in 30 days

# Collapse the first 2 categories
df['readmitted2'] = np.where(df['readmitted'] == '<30', 1, 0)

# Check the new variable with collapsed categories:
ft_readmitted2 = df['readmitted2'].value_counts()

# Calculate the percentage and add it as a new column
ft_readmitted2_percent = ft_readmitted2.to_frame(name='Frequency')
ft_readmitted2_percent['Percentage'] = (ft_readmitted2_percent['Frequency'] / len(df)) * 100
print(ft_readmitted2_percent)

             Frequency  Percentage
readmitted2                       
0                90409   88.840084
1                11357   11.159916


In [9]:
# sample size
len(df)
print('Rows: {}'.format(len(df)))

Rows: 101766


In [10]:
# Evaluation of sample size - with 11,357 cases in the smaller category in outcome (readmitted within 30 days)
# and about 40 useful predictors, that's over 280 cases per predictor which is more than enough for the recommended
# size for a logistic regression (at least 10-15 cases in smaller category outcome per predictor), and sample size seems
# large enough for some other models such as SVM, although may not be big enough for multilayer perceptron. We shall see...

In [11]:
# Turning to look at the independence of cases, I see that some patients are in the dataset more than
# once because they had more than 1 hospital readmission. That's not ideal - that means some cases aren't
# independent of one another which violates assumptions behind some analyses like logistical regression
# Lets assess how bad that is by seeing how many patients have more than 1 hospitalization

# Check the frequency of each patient:
ft_pts = df['patient_nbr'].value_counts()
ft_duplicate_pts = ft_pts.to_frame(name='Frequency')
ft_duplicate_pts = ft_duplicate_pts[ft_duplicate_pts['Frequency'] > 1]

print(ft_duplicate_pts)
print('Duplicate pts account for ' + str(ft_duplicate_pts['Frequency'].sum()) + ' rows in dataset')

             Frequency
patient_nbr           
88785891            40
43140906            28
1660293             23
88227540            23
23199021            23
...                ...
239535               2
44895969             2
85504905             2
39987954             2
92028474             2

[16773 rows x 1 columns]
Duplicate pts account for 47021 rows in dataset


In [112]:
# Out of a dataset with >101,000 rows, looks like there are 16,000 patients in this dataset who have more than 1 
# hospitalization, some with up to 40 hospitalizations.  These duplicates account for 47,000 rows, which is almost half
# the dataset.  This is a large violation of independence of cases. If this were a more professional analysis, I 
# believe I'd need to use MLM to account for this but we'll proceed anyway since this project is for my personal practice

In [12]:
# Checking continuous predictors for missingness, out of range/incorrect values, normality
continuous_variables = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']

def check_missingness(var):
    print('Missing in {}: {}'.format(var, df[var].isnull().sum()))

def check_for_invalid_strings(var):
    print('{} contains string: {}'.format(var, any(df[var].apply(lambda x: isinstance(x, str)))))

def histogram(var):
    df[var].hist(bins=10, figsize=(6, 4))
    plt.title(var)
    plt.show()

for col in continuous_variables:
    check_missingness(col)
    check_for_invalid_strings(col) 
    print(df[col].describe()) # check min, max, mean, quartiles, std
    # histogram(col) # turning off to avoid uploading too many images to github
    print()

Missing in time_in_hospital: 0
time_in_hospital contains string: False
count    101766.000000
mean          4.395987
std           2.985108
min           1.000000
25%           2.000000
50%           4.000000
75%           6.000000
max          14.000000
Name: time_in_hospital, dtype: float64

Missing in num_lab_procedures: 0
num_lab_procedures contains string: False
count    101766.000000
mean         43.095641
std          19.674362
min           1.000000
25%          31.000000
50%          44.000000
75%          57.000000
max         132.000000
Name: num_lab_procedures, dtype: float64

Missing in num_procedures: 0
num_procedures contains string: False
count    101766.000000
mean          1.339730
std           1.705807
min           0.000000
25%           0.000000
50%           1.000000
75%           2.000000
max           6.000000
Name: num_procedures, dtype: float64

Missing in num_medications: 0
num_medications contains string: False
count    101766.000000
mean         16.021844


In [13]:
df['num_lab_procedures'].describe()

count    101766.000000
mean         43.095641
std          19.674362
min           1.000000
25%          31.000000
50%          44.000000
75%          57.000000
max         132.000000
Name: num_lab_procedures, dtype: float64

In [115]:
""" Summary: 
time_in_hospital - skewed distribution but some variance
num_lab_procedures - looks good
num_procedures - oof, weird distribution
num_medications - looks good
number_outpatient - very skewed distribution. Almost all have 0
number_emergency - very skewed distribution. almost all have 0
number_inpatient - very skewed distribution
number_diagnoses - skewed distribution but some variance
"""

# Based on these results, I'm gonna plan to exclude num_procedures, number_outpatient, 
# number_emergency, and number_inpatient from logistic regression but leave in for other
# analyses that aren't as picky
# If I had more time, and logistic regression was my final goal, I'd do some transformations
# on the other variables to produce a more normal distribution, but since stronger models
# are less picky, I'll leave them be.

' Summary: \ntime_in_hospital - skewed distribution but some variance\nnum_lab_procedures - looks good\nnum_procedures - oof, weird distribution\nnum_medications - looks good\nnumber_outpatient - very skewed distribution. Almost all have 0\nnumber_emergency - very skewed distribution. almost all have 0\nnumber_inpatient - very skewed distribution\nnumber_diagnoses - skewed distribution but some variance\n'

In [14]:
# Checking categorical predictors for missingness, out of range, 
categorical_predictors = ['race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed']

def frequency_table(var):
    frequency_table = df[var].value_counts()
    # Calculate the percentage and add it as a new column
    ft_percent = frequency_table.to_frame(name='Frequency')
    ft_percent['Percentage'] = (ft_percent['Frequency'] / len(df)) * 100
    print(ft_percent)

for col in categorical_predictors:
    print('------ ' + col + ' ------')
    check_missingness(col)
    frequency_table(col)


------ race ------
Missing in race: 0
                 Frequency  Percentage
race                                  
Caucasian            76099   74.778413
AfricanAmerican      19210   18.876639
?                     2273    2.233555
Hispanic              2037    2.001651
Other                 1506    1.479866
Asian                  641    0.629876
------ gender ------
Missing in gender: 0
                 Frequency  Percentage
gender                                
Female               54708   53.758623
Male                 47055   46.238429
Unknown/Invalid          3    0.002948
------ age ------
Missing in age: 0
          Frequency  Percentage
age                            
[70-80)       26068   25.615628
[60-70)       22483   22.092840
[50-60)       17256   16.956547
[80-90)       17197   16.898571
[40-50)        9685    9.516931
[30-40)        3775    3.709490
[90-100)       2793    2.744532
[20-30)        1657    1.628245
[10-20)         691    0.679009
[0-10)          161    0.

In [117]:
# Results of looking at categorical variables
"""
race - 2% are missing, currently coded with "?", and 1.5 are "Other"
74% white, 19% AA. Need to recode as numeric, not strings. 
Could collapse into 2 categories "white, POC" for logistic regression but retain diversity for other models. 

gender - reasonable split with a small number UNK. Need to recode as numeric, not strings.

age - no missing data, babies and children are a fraction of a percent (less than 150 rows). Most participants are age 40+. 
Consider trimming dataset to only age 12+ or age 18+, as babies and young children are poorly represented in dataset
and have such different health needs that any model based on this data would be unlikely to predict their readmission 
very well.

weight: 96% of data is missing - remove this variable from all analyses

admission_type_id: various types of missing (5, 6, 8) make up about 10-11%
53% emergency, 18% urgent, 18% elective, and fractions of a percent were newborn or trauma admissions. 
Would it make theoretical sense to lump trauma center in with emergency or urgent? Leave as is for now.

discharge_disposition_id = categories 1 and 6 (discharged home) make up 71%, of the rest, some 18% discharged 
to higher level of care. Some discharged as expired, and those cases should be removed from the analysis, 
since we already know patients who passed away aren't going to be readmitted

lab results - most are 98%+ in one category, so not much diversity here for model to learn from.

some labs examide and citoglipton were 100% no, so can exclude for all analyses since offer no variety
max_glu_serum and A1Cresult are mostly missing, so will also exclude from all analyses

change = 53% had no change to their meds, and 46% did
diabetesMed = 77% were on diabetes meds, and 23% not
"""

'\nrace - 2% are missing, currently coded with "?", and 1.5 are "Other"\n74% white, 19% AA. Need to recode as numeric, not strings. \nCould collapse into 2 categories "white, POC" for logistic regression but retain diversity for other models. \n\ngender - reasonable split with a small number UNK. Need to recode as numeric, not strings.\n\nage - no missing data, babies and children are a fraction of a percent (less than 150 rows). Most participants are age 40+. \nConsider trimming dataset to only age 12+ or age 18+, as babies and young children are poorly represented in dataset\nand have such different health needs that any model based on this data would be unlikely to predict their readmission \nvery well.\n\nweight: 96% of data is missing - remove this variable from all analyses\n\nadmission_type_id: various types of missing (5, 6, 8) make up about 10-11%\n53% emergency, 18% urgent, 18% elective, and fractions of a percent were newborn or trauma admissions. \nWould it make theoretical

In [15]:
# Based on univariate look at predictors above, going to do the following:
# Remove babies and young children who have different health needs and are a tiny fraction of dataset
# Remove anyone who was reported as having died at discharge
df = df[df['age'] != '[0-10)']
df = df[df['discharge_disposition_id'] != 11]
df = df[df['discharge_disposition_id'] != 19]
df = df[df['discharge_disposition_id'] != 20]
df = df[df['discharge_disposition_id'] != 21]
#df = df[df['discharge_disposition_id'] != 11 or df['discharge_disposition_id'] != 19 or df['discharge_disposition_id'] != 20 or df['discharge_disposition_id'] != 21]

In [16]:
# sample size
len(df)
print('New number of rows in dataset: {}'.format(len(df)))
for col in ['age', 'discharge_disposition_id']:
    print('------ ' + col + ' ------')
    check_missingness(col)
    frequency_table(col)

New number of rows in dataset: 99954
------ age ------
Missing in age: 0
          Frequency  Percentage
age                            
[70-80)       25562   25.573764
[60-70)       22185   22.195210
[50-60)       17102   17.109871
[80-90)       16706   16.713688
[40-50)        9626    9.630430
[30-40)        3765    3.766733
[90-100)       2668    2.669228
[20-30)        1650    1.650759
[10-20)         690    0.690318
------ discharge_disposition_id ------
Missing in discharge_disposition_id: 0
                          Frequency  Percentage
discharge_disposition_id                       
1                             60089   60.116654
3                             13954   13.960422
6                             12900   12.905937
18                             3691    3.692699
2                              2127    2.127979
22                             1993    1.993917
5                              1183    1.183544
25                              979    0.979451
4                

In [17]:
# recode variables as numeric or np.nan and collapse categories as needed
# collapse race categories - Latinx, Asian, and other pts are so underrepresented in this data
race_dict = {'Caucasian': 0, 'AfricanAmerican': 1, 'Hispanic': 1, 'Asian': 1, 'Other': 1, '?': np.nan}
df['race2'] = df['race'].map(race_dict)

gender_dict = {'Male': 0, 'Female': 1, 'Unknown/Invalid': np.nan}
df['gender2'] = df['gender'].map(gender_dict)

age_dict = {'[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35, '[40-50)': 45, '[50-60)': 55, '[60-70)': 65, '[70-80)': 75, '[80-90)': 85, '[90-100)': 95}
df['age2'] = df['age'].map(age_dict)

# collapse categories in discharge_disposition_id such that:
#    1: discharged to equivalent or higher level of care (e.g., inpatient)
#    0: discharged home or to lower level of care (e.g., outpatient)
#    2: left against medical advice
#    'expired': expired (shouldn't be any left in dataset)
#    'unknown': various unknowns, or not enough detail to know if higher/lower level of care
discharge_disposition_id_dict = {1: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 2, 8: 0, 9: 1, 10: 1, 11: 3, 12: 0, 13: 0, 14: 1, 15: 1, 16: 0, 17: 0, 18: np.nan, 19: 'expired', 20: 'expired', 21: 'expired', 22: 1, 23: 1, 24: 1, 25: np.nan, 26: np.nan, 27: 1, 28: 1, 29: 1, 30: np.nan}
df['discharge_disposition_id2'] = df['discharge_disposition_id'].map(discharge_disposition_id_dict)

# collapse unknown categories in admission_type_id and anyone with admission_type_id = 4 (newborn) whose age was not 0-10
admission_type_id_dict = {1: 1, 2: 2, 3: 3, 4: np.nan, 5: np.nan, 6: np.nan, 7: 7, 8: np.nan}
df['admission_type_id2'] = df['admission_type_id'].map(admission_type_id_dict)

# Check on results
for col in ['race2', 'gender2', 'age2', 'discharge_disposition_id2', 'admission_type_id2']:
    print('------ ' + col + ' ------')
    frequency_table(col)

------ race2 ------
       Frequency  Percentage
race2                       
0.0        74710   74.744382
1.0        23006   23.016588
------ gender2 ------
         Frequency  Percentage
gender2                       
1.0          53779   53.803750
0.0          46172   46.193249
------ age2 ------
      Frequency  Percentage
age2                       
75        25562   25.573764
65        22185   22.195210
55        17102   17.109871
85        16706   16.713688
45         9626    9.630430
35         3765    3.766733
95         2668    2.669228
25         1650    1.650759
15          690    0.690318
------ discharge_disposition_id2 ------
                           Frequency  Percentage
discharge_disposition_id2                       
0                              73524   73.557837
1                              21138   21.147728
2                                622    0.622286
------ admission_type_id2 ------
                    Frequency  Percentage
admission_type_id2             

In [18]:
# Recode lab variables as numeric
lab_dict = {'No': 0, 'Down': 1, 'Steady': 2, 'Up': 3}

labs = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']
for col in labs:
    df[f'{col}2'] = df[col].map(lab_dict)

column_names = df.columns.tolist()
print("Column Names:", column_names)

df.head()

Column Names: ['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted', 'readmitted2', 'race2', 'gender2', 'age2', 'discharge_disposition_id2', 'admission_type_id2', 'metformin2', 'repaglinide2', 'nateglinide2', 'chlorpropamide2', 'glimepir

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,acarbose2,miglitol2,troglitazone2,tolazamide2,insulin2,glyburide-metformin2,glipizide-metformin2,glimepiride-pioglitazone2,metformin-rosiglitazone2,metformin-pioglitazone2
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,0,0,0,0,3,0,0,0,0,0
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,0,0,0,0,0,0,0,0,0,0
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,0,0,0,0,3,0,0,0,0,0
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,0,0,0,0,2,0,0,0,0,0
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,...,0,0,0,0,2,0,0,0,0,0


In [19]:
# recode change and diabetesMed as numeric
change_dict = {'No': 0, 'Ch': 1}
df['change2'] = df['change'].map(change_dict)

diabetesMed_dict = {'No': 0, 'Yes': 1}
df['diabetesMed2'] = df['diabetesMed'].map(diabetesMed_dict)

In [20]:
# Drop columns with no variability, tons of missing data, recoded, or no theoretical connection to hospital readmission
# or those that were recoded
drop_cols = ['encounter_id', 'patient_nbr', 'weight', 'admission_source_id', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'examide', 'citoglipton', 'max_glu_serum', 'A1Cresult', 'admission_type_id', 'discharge_disposition_id', 'race', 'gender', 'age', 'readmitted']
df.drop(drop_cols, axis=1, inplace=True) 
df.drop(labs, axis=1, inplace=True) 
df.drop(['change', 'diabetesMed'], axis=1, inplace=True) 

column_names = df.columns.tolist()
print("Column Names:", column_names)

Column Names: ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'readmitted2', 'race2', 'gender2', 'age2', 'discharge_disposition_id2', 'admission_type_id2', 'metformin2', 'repaglinide2', 'nateglinide2', 'chlorpropamide2', 'glimepiride2', 'acetohexamide2', 'glipizide2', 'glyburide2', 'tolbutamide2', 'pioglitazone2', 'rosiglitazone2', 'acarbose2', 'miglitol2', 'troglitazone2', 'tolazamide2', 'insulin2', 'glyburide-metformin2', 'glipizide-metformin2', 'glimepiride-pioglitazone2', 'metformin-rosiglitazone2', 'metformin-pioglitazone2', 'change2', 'diabetesMed2']


In [21]:
# re-check what percent of data in these columns are missing
def check_missingness_percent(var):
    percent_missing = (df[var].isnull().sum() / len(df)) * 100
    print('Missing in {}: {}%'.format(var, percent_missing))

for col in column_names:
    check_missingness_percent(col)
    #frequency_table(col)

Missing in time_in_hospital: 0.0%
Missing in num_lab_procedures: 0.0%
Missing in num_procedures: 0.0%
Missing in num_medications: 0.0%
Missing in number_outpatient: 0.0%
Missing in number_emergency: 0.0%
Missing in number_inpatient: 0.0%
Missing in number_diagnoses: 0.0%
Missing in readmitted2: 0.0%
Missing in race2: 2.239029953778738%
Missing in gender2: 0.003001380635092142%
Missing in age2: 0.0%
Missing in discharge_disposition_id2: 4.6721491886267685%
Missing in admission_type_id2: 10.2367088860876%
Missing in metformin2: 0.0%
Missing in repaglinide2: 0.0%
Missing in nateglinide2: 0.0%
Missing in chlorpropamide2: 0.0%
Missing in glimepiride2: 0.0%
Missing in acetohexamide2: 0.0%
Missing in glipizide2: 0.0%
Missing in glyburide2: 0.0%
Missing in tolbutamide2: 0.0%
Missing in pioglitazone2: 0.0%
Missing in rosiglitazone2: 0.0%
Missing in acarbose2: 0.0%
Missing in miglitol2: 0.0%
Missing in troglitazone2: 0.0%
Missing in tolazamide2: 0.0%
Missing in insulin2: 0.0%
Missing in glyburid

In [22]:
# since the variables with missing data (race2, gender2, discharge_disposition_id2 and admission_type_id2)
# are all categorical, seems like a reasonable method of imputation is method is to replace with mode

# make a copy of original data
df_copy = df.copy()

# use SimpleImputer with some parameters
imp = SimpleImputer(strategy="most_frequent", missing_values=np.nan)
nd_array = imp.fit_transform(df_copy)

# since the fit_transform method of SimpleImputer returns a numpy.ndarray, convert back to a data frame and retrieve
# the column names
df_no_missing = pd.DataFrame(nd_array)
df_no_missing.columns = df_copy.columns

# Check/compare
df_no_missing.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted2,race2,...,troglitazone2,tolazamide2,insulin2,glyburide-metformin2,glipizide-metformin2,glimepiride-pioglitazone2,metformin-rosiglitazone2,metformin-pioglitazone2,change2,diabetesMed2
0,3,59,0,18,0,0,0,9,0,0.0,...,0,0,3,0,0,0,0,0,1,1
1,2,11,5,13,2,0,1,6,0,1.0,...,0,0,0,0,0,0,0,0,0,1
2,2,44,1,16,0,0,0,7,0,0.0,...,0,0,3,0,0,0,0,0,1,1
3,1,51,0,8,0,0,0,5,0,0.0,...,0,0,2,0,0,0,0,0,1,1
4,3,31,6,16,0,0,0,9,0,0.0,...,0,0,2,0,0,0,0,0,0,1


In [23]:
df.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted2,race2,...,troglitazone2,tolazamide2,insulin2,glyburide-metformin2,glipizide-metformin2,glimepiride-pioglitazone2,metformin-rosiglitazone2,metformin-pioglitazone2,change2,diabetesMed2
1,3,59,0,18,0,0,0,9,0,0.0,...,0,0,3,0,0,0,0,0,1,1
2,2,11,5,13,2,0,1,6,0,1.0,...,0,0,0,0,0,0,0,0,0,1
3,2,44,1,16,0,0,0,7,0,0.0,...,0,0,3,0,0,0,0,0,1,1
4,1,51,0,8,0,0,0,5,0,0.0,...,0,0,2,0,0,0,0,0,1,1
5,3,31,6,16,0,0,0,9,0,0.0,...,0,0,2,0,0,0,0,0,0,1


In [24]:
# There should be no missing data in the new version and the highest categories should have increased slightly
def frequency_table2(var):
    frequency_table = df_no_missing[var].value_counts()
    # Calculate the percentage and add it as a new column
    ft_percent = frequency_table.to_frame(name='Frequency')
    ft_percent['Percentage'] = (ft_percent['Frequency'] / len(df)) * 100
    print(ft_percent)

for col in ['race2', 'gender2', 'discharge_disposition_id2', 'admission_type_id2']:
    percent_missing = (df_no_missing[col].isnull().sum() / len(df_no_missing)) * 100
    print('Missing in {}: {}%'.format(col, percent_missing))
    frequency_table2(col)
    print()
    print()

Missing in race2: 0.0%
       Frequency  Percentage
race2                       
0.0        76948   76.983412
1.0        23006   23.016588


Missing in gender2: 0.0%
         Frequency  Percentage
gender2                       
1.0          53782   53.806751
0.0          46172   46.193249


Missing in discharge_disposition_id2: 0.0%
                           Frequency  Percentage
discharge_disposition_id2                       
0                              78194   78.229986
1                              21138   21.147728
2                                622    0.622286


Missing in admission_type_id2: 0.0%
                    Frequency  Percentage
admission_type_id2                       
1.0                     63012   63.040999
3.0                     18723   18.731617
2.0                     18201   18.209376
7.0                        18    0.018008




In [25]:
# explore multivariate relationships
outcome = 'readmitted2'
categorical_predictors = ['race2', 'gender2', 'acetohexamide2', 'tolbutamide2', 'troglitazone2', 'glipizide-metformin2', 'glimepiride-pioglitazone2', 'metformin-rosiglitazone2', 'metformin-pioglitazone2', 'change2', 'diabetesMed2', 'discharge_disposition_id2', 'admission_type_id2', 'metformin2', 'repaglinide2', 'nateglinide2', 'chlorpropamide2', 'glimepiride2', 'glipizide2', 'glyburide2', 'pioglitazone2', 'rosiglitazone2', 'acarbose2', 'miglitol2', 'tolazamide2', 'insulin2', 'glyburide-metformin2']
ordinal_predictors = ['age2'] # could be used as categorical or continuous given original data's age categories
continuous_predictors = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']

In [41]:
# chi-squared tests for independence between non-binary categorical predictors and outcome
for col in categorical_predictors:
    # print(col)
    contingency_table = pd.crosstab(df_no_missing[outcome], df_no_missing[col])
    # print(tabulate(contingency_table, headers='keys', tablefmt='fancy_grid'))
    chi2, p, _, _ = chi2_contingency(contingency_table)
    # print('Chi-squared statistic: ' + str(round(chi2, 3)) + ', p-value: ' + str(round(p, 3)))
    # print()
# Note: Turned off these lines due to large amount of output in github

In [27]:
# Above, we can see significant relationships between the outcome (readmission) and some predictors including
# change in meds, diabetes meds, discharge disposition, admission type and a few of the labs


In [28]:
# independent 2-sample t-tests between continuous predictors and outcome

for col in continuous_predictors:
    print(col)
    # Separate the data into two groups based on the binary outcome
    group1 = df_no_missing[df_no_missing[outcome] == 1][col].to_numpy().tolist()
    group2 = df_no_missing[df_no_missing[outcome] == 0][col].to_numpy().tolist()
    # Perform independent two-sample t-test
    t_stat, p_value = ttest_ind(group1, group2)
    # Display the results
    print('t-statistic: ' + str(round(t_stat, 3)) + ' p-value: ' + str(round(p_value, 3)))


time_in_hospital
t-statistic: 14.322 p-value: 0.0
num_lab_procedures
t-statistic: 7.386 p-value: 0.0
num_procedures
t-statistic: -3.436 p-value: 0.001
num_medications
t-statistic: 12.708 p-value: 0.0
number_outpatient
t-statistic: 5.99 p-value: 0.0
number_emergency
t-statistic: 19.233 p-value: 0.0
number_inpatient
t-statistic: 53.691 p-value: 0.0
number_diagnoses
t-statistic: 16.283 p-value: 0.0


In [29]:
# Should do more checks here on variance, but this would suggest that all continuous predictors
# are significantly correlated with readdmission

In [30]:
# check correlations between all continuous predictors and each other

# make dataset with only continous predictors
df_continuous = df_no_missing[continuous_predictors].copy()

# df_continuous.head()

# Calculate the correlation matrix
correlation_matrix = df_continuous.corr()

# Create a styled correlation table with color highlighting
styled_corr_table = correlation_matrix.style.background_gradient(cmap='coolwarm')

# Display the styled correlation table
styled_corr_table

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
time_in_hospital,1.000000,0.319863,0.189493,0.463566,-0.009758,-0.009849,0.073582,0.219716
num_lab_procedures,0.319863,1.000000,0.051531,0.265149,-0.007773,-0.001482,0.039114,0.150458
num_procedures,0.189493,0.051531,1.000000,0.380891,-0.025502,-0.038849,-0.067845,0.067189
num_medications,0.463566,0.265149,0.380891,1.000000,0.045231,0.013638,0.064132,0.257704
number_outpatient,-0.009758,-0.007773,-0.025502,0.045231,1.000000,0.091724,0.107705,0.094101
number_emergency,-0.009849,-0.001482,-0.038849,0.013638,0.091724,1.000000,0.266970,0.055505
number_inpatient,0.073582,0.039114,-0.067845,0.064132,0.107705,0.266970,1.000000,0.103515
number_diagnoses,0.219716,0.150458,0.067189,0.257704,0.094101,0.055505,0.103515,1.000000


In [31]:
# None of these variables are highly correlated to each other, so that's fine. No multicollinearity

In [32]:
# check age as if it were categorical
contingency_table = pd.crosstab(df_no_missing[outcome], df_no_missing['age2'])
print(tabulate(contingency_table, headers='keys', tablefmt='fancy_grid'))
chi2, p, _, _ = chi2_contingency(contingency_table)
print('Chi-squared statistic: ' + str(round(chi2, 3)) + ', p-value: ' + str(round(p, 3)))
print()

# check age as if it were continuous
group1 = df_no_missing[df_no_missing[outcome] == 1]['age2'].to_numpy().tolist()
group2 = df_no_missing[df_no_missing[outcome] == 0]['age2'].to_numpy().tolist()
t_stat, p_value = ttest_ind(group1, group2)
print('t-statistic: ' + str(round(t_stat, 3)) + ' p-value: ' + str(round(p_value, 3)))

╒═══════════════╤══════╤══════╤══════╤══════╤═══════╤═══════╤═══════╤═══════╤══════╕
│   readmitted2 │   15 │   25 │   35 │   45 │    55 │    65 │    75 │    85 │   95 │
╞═══════════════╪══════╪══════╪══════╪══════╪═══════╪═══════╪═══════╪═══════╪══════╡
│             0 │  650 │ 1414 │ 3341 │ 8599 │ 15434 │ 19683 │ 22493 │ 14628 │ 2358 │
├───────────────┼──────┼──────┼──────┼──────┼───────┼───────┼───────┼───────┼──────┤
│             1 │   40 │  236 │  424 │ 1027 │  1668 │  2502 │  3069 │  2078 │  310 │
╘═══════════════╧══════╧══════╧══════╧══════╧═══════╧═══════╧═══════╧═══════╧══════╛
Chi-squared statistic: 114.078, p-value: 0.0

t-statistic: 6.087 p-value: 0.0


In [33]:
# Looks like age has a significant relationship with readmission either way

In [34]:
column_names = df_no_missing.columns.tolist()
print("Column Names:", column_names)

Column Names: ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'readmitted2', 'race2', 'gender2', 'age2', 'discharge_disposition_id2', 'admission_type_id2', 'metformin2', 'repaglinide2', 'nateglinide2', 'chlorpropamide2', 'glimepiride2', 'acetohexamide2', 'glipizide2', 'glyburide2', 'tolbutamide2', 'pioglitazone2', 'rosiglitazone2', 'acarbose2', 'miglitol2', 'troglitazone2', 'tolazamide2', 'insulin2', 'glyburide-metformin2', 'glipizide-metformin2', 'glimepiride-pioglitazone2', 'metformin-rosiglitazone2', 'metformin-pioglitazone2', 'change2', 'diabetesMed2']


In [35]:
# split the df into x and y
y = df_no_missing[[outcome]] # get the outcome variable into a new df called y
y.head() # check it

,readmitted2
0,0
1,0
2,0
3,0
4,0


In [36]:
predictors = list(df_no_missing.columns) # get a list of columns
predictors.remove(outcome) # remove the outcome
predictors # check it
x = df_no_missing[predictors] # x is the df with only predictor columns
x.head() # check it

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race2,gender2,...,troglitazone2,tolazamide2,insulin2,glyburide-metformin2,glipizide-metformin2,glimepiride-pioglitazone2,metformin-rosiglitazone2,metformin-pioglitazone2,change2,diabetesMed2
0,3,59,0,18,0,0,0,9,0.0,1.0,...,0,0,3,0,0,0,0,0,1,1
1,2,11,5,13,2,0,1,6,1.0,1.0,...,0,0,0,0,0,0,0,0,0,1
2,2,44,1,16,0,0,0,7,0.0,0.0,...,0,0,3,0,0,0,0,0,1,1
3,1,51,0,8,0,0,0,5,0.0,0.0,...,0,0,2,0,0,0,0,0,1,1
4,3,31,6,16,0,0,0,9,0.0,0.0,...,0,0,2,0,0,0,0,0,0,1


In [37]:
# Split into default 25% test and 75% training
# add a seed to the randomizer so the split will be consistent/reproducible
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1234)

In [38]:
# For logistic regression model or other picky models, further trim out predictors that are 
# have low variance or no relationship to outcome
columns_to_keep = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'age2', 'discharge_disposition_id2', 'admission_type_id2', 'metformin2', 'repaglinide2', 'pioglitazone2', 'insulin2', 'change2', 'diabetesMed2']
x_train_reduced = x_train[columns_to_keep]
x_test_reduced = x_test[columns_to_keep]
x_train_reduced.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age2,discharge_disposition_id2,admission_type_id2,metformin2,repaglinide2,pioglitazone2,insulin2,change2,diabetesMed2
54162,5,44,3,8,1,0,1,8,65,0,2.0,2,0,0,0,0,1
92909,4,13,0,20,1,5,4,9,55,0,1.0,0,0,0,2,0,1
81606,7,34,2,33,5,0,0,9,75,0,3.0,2,0,0,0,0,1
47304,4,43,0,10,2,0,1,7,65,0,1.0,0,0,0,3,1,1
27408,2,42,0,12,0,0,0,5,65,0,1.0,0,0,0,2,1,1


In [39]:
x_test_reduced = x_test[columns_to_keep]

In [ ]:
# If models struggle, I could continue with additional data exploration and cleaning recommended for 
# logistic regression including checking outliers, centering continuous variables / transforming some to 
# reduce skew, and strategies to deal with imbalanced categories among categorical variables
# However, since logistic regression is not my only model, and other models handle messy data much better
# let's proceed as is and see how these models do.

In [40]:
# export datasets to be used in models
x_train.to_csv('./x_train.csv')
y_train.to_csv('./y_train.csv')
x_test.to_csv('./x_test.csv')
y_test.to_csv('./y_test.csv')
x_train_reduced.to_csv('./x_train_reduced.csv')
x_test_reduced.to_csv('./x_test_reduced.csv')